##### Imports & Constants

In [ ]:
import pathlib
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
import sklearn.model_selection as ms
import sklearn.linear_model as lm
import sklearn.ensemble as ske
import sklearn.metrics as skm

FINAL_DATA_PATH = pathlib.Path('..') / 'data' / 'final' / 'final_data.csv'
PCA_DATA_PATH = pathlib.Path('..') / 'data' / 'final' / 'pca_data.csv'

final_data = pd.read_csv(FINAL_DATA_PATH)
pca_data = pd.read_csv(PCA_DATA_PATH)

X, y = final_data.drop('position', axis=1), final_data['position']
pca_X, pca_y = pca_data, final_data['position']

# Split data into train and test sets
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
pca_X_train, pca_X_test, pca_y_train, pca_y_test = ms.train_test_split(pca_X, pca_y, test_size=0.2, random_state=42, shuffle=True)

# Split train data into train and validation sets
X_train, X_val, y_train, y_val = ms.train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

X_train_features_count = X_train.shape[1]
pca_X_train_features_count = pca_X_train.shape[1]

##### Inital Model

In [ ]:
rf = ske.RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

""" 
0 -> ST
1 -> LW
2 -> RW
3 -> CM
4 -> RB
5 -> LB
6 -> CB
"""

# Metrics
print(skm.classification_report(y_test, y_pred))

In [ ]:
feature_importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
sns.barplot(x=feature_importance, y=feature_importance.index)

##### Inital Model (PCA)

In [ ]:
rf = ske.RandomForestClassifier(n_estimators=10, random_state=42)
rf.fit(pca_X_train, pca_y_train)
y_pred = rf.predict(pca_X_test)

""" 
0 -> ST
1 -> LW
2 -> RW
3 -> CM
4 -> RB
5 -> LB
6 -> CB
"""

# Metrics
print(skm.classification_report(pca_y_test, y_pred))


In [ ]:
feature_importance = pd.Series(rf.feature_importances_, index=pca_X.columns).sort_values(ascending=False)
sns.barplot(x=feature_importance, y=feature_importance.index)

In [ ]:
lr = lm.LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Metrics
print(skm.classification_report(pca_y_test, y_pred))

## Tensorflow softmax ANN

In [ ]:
""" 
This model was tested with different number of layers and neurons per layer.
The best result was achieved with the following configuration:
    - 5 layers
    - 256 neurons in the first layer
    - 128 neurons in the second layer
    - 64 neurons in the third layer
    - 32 neurons in the fourth layer
    - 7 neurons in the fifth layer

The model was trained with 100 epochs and a batch size of 32.
"""


import tensorflow as tf

# read model

model = None
try:
    model = tf.keras.models.load_model('model.h5')
except:
    pass

if not model:
    
    input_size = X_train_features_count

    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(input_size, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ])

    # compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # train model
    model.fit(
        X_train,
        y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val)
    )
    

# evaluate model
model.evaluate(X_test, y_test, verbose=2)

In [ ]:
# descirbe model
model.summary()

# save model
model.save('model.h5')


In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

try:
    model = pickle.load(open('randomforest_model.sav', 'rb'))
except:
    model = None
    
if not model:
    
    param_grid = {
        'n_estimators': [25, 50, 100, 150],
        'max_features': ['sqrt', 'log2', None],
        'max_depth': [6, 9, 12, 24, None],
        'max_leaf_nodes': [6, 9, 12, 24, None],
    }

    model = RandomForestClassifier()

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        scoring='accuracy',
        n_jobs=15,
        verbose=1
    )

    grid_search.fit(X_train, y_train)
    
    best_n_estimators = grid_search.best_params_['n_estimators']
    best_max_depth = grid_search.best_params_['max_depth']
    best_max_features = grid_search.best_params_['max_features']
    best_max_leaf_nodes = grid_search.best_params_['max_leaf_nodes']

    best_model = RandomForestClassifier(
        n_estimators=best_n_estimators,
        max_depth=best_max_depth,
        max_features=best_max_features,
        max_leaf_nodes=best_max_leaf_nodes,
    )
    best_model.fit(X_train, y_train)

    y_pred_test = best_model.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred_test)
    print("Test Accuracy:", test_accuracy)
    
    # Save model
    pickle.dump(best_model, open('randomforest_model.sav', 'wb'))

In [ ]:
print(f"{model.n_estimators=}")
print(f"{model.max_depth=}")
print(f"{model.max_features=}")
print(f"{model.max_leaf_nodes=}")

model.score(X_test, y_test)

In [ ]:
# display confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')